# Merge And Graph
This script loads all the measures tables generated, calculates the bootstraps and plots it


### Imports + Global Setup

In [8]:
from __future__ import print_function, division
%matplotlib inline

import sys 

def add_module_path(module_path):
    if module_path not in sys.path:
        sys.path.append(module_path)

add_module_path("../colour/")
add_module_path("../PyEDF/")

import numpy as np
import pandas as pd
import re
import os
import warnings
import math

from matplotlib import pyplot as plt
import scikits.bootstrap as boot

from operator import itemgetter

# import pyedf
# import conf

pd.options.display.mpl_style = 'default'

plt.rcParams['figure.figsize'] = (16.0, 10.0)
plt.set_cmap('gnuplot2')



## Global Constants

In [9]:
# path = './DataDummy'
#path = r'G:\Miguel\Research\Research-Current\CMOS\Git Experiment Only\Logs (For test analysis)'
#path = r'G:\Miguel\Research\Research-Current\CMOS\pilot data'
path = ".\processedData"
#participant_ids = [ 'None', 'Johanas' ]
participant_ids = [  '15FM24',
                     '21MDC20',
                     '3MW2421',
                     '22F',
                     '14JG23',
                     '1MQ2201',
                     '2ML3603',
                     '8FC02',
                     '5MS3205',
                     '9FH04',
                     '1MW2207',
                     '12FRH06',
                     '6MDH09',
                     '13MM08',
                     '12JR11',
                     '10F',
                     '17MRW13',
                     '12F',
                     'MPS15',
                     '2FQ2014',
                     '5MS17',
                     '2FS2116',
                     '18MDM19',
                     '18F']
condition_ids = ['e', 'c', 's', 'n']
#condition_ids = ['e', 'c']
mediaObjectsPerDocument = 20 # this comes from how the documents were set up.
mediaObjectsScotland = 4 # this comes from how the documents were set up.

## Loading Calculated Data 

In [10]:
pathEyeResults = path+'\\'+"fixationAnalysisResults_pandas.pddf"
pathQuestionnaireResults = path+'\\'+'TLX_and_Questions_AnalysisResults_pandas.pddf'
pathInteractionResults = path+'\\'+'interactionAnalysisResults_pandas.pddf'

frameEye = pd.read_pickle(pathEyeResults)
frameQuestionnaire = pd.read_pickle(pathQuestionnaireResults)
frameInteraction = pd.read_pickle(pathInteractionResults)

#frameEyeFiltered = frameEye

allData = pd.concat([frameEye,frameQuestionnaire,frameInteraction], axis = 1)
#allData = frameEye.append([frameQuestionnaire,frameInteraction])







In [11]:
pd.set_option('display.max_columns', None)
frameEye[:5]
allData


,M10,M11,M12,M12m,M12o,M12t,M13m,M13o,M13t,M3,M3m,M3o,M3t,M4,M4m,M4o,M4perSec,M4t,M5,M5m,M5o,M5t,M6,M6m,M6o,M6t,M7,M8,M9,V10t,country,experimentNumber,orderNumber,training,type,userId,M1,M18,M19,M2,M20,M21,M22,M23,M24,M25,M26,M29,M2country,M30,M30country,country,experimentNumber,orderNumber,training,type,userId,M14,M15,M15m,M15o,M15t,M16,M16m,M16o,M16t,M17,country,experimentNumber,orderNumber,training,type,userId
0,12,280.572,217.356,15.936,22.044,179.376,0.073318,0.101419,0.825264,0.202568,0.227657,0.198595,0.201094,1073,70,111,3.824330,892,157.542478,102.111412,88.310909,151.510704,0.058970,0.049920,0.054000,0.057422,21,1.05,541.613187,13,3,126,24,False,e,15FM24,0.500,1,1,0.250,1,1,1,1,19,11,9,4,NaN,0.250,NaN,0,125,24,False,e,15FM24,16073,1,0,1,0,282589,0,53137,229452,5999.696358,3,126,24,False,e,15FM24
1,0,47.832,35.136,0.000,7.356,27.780,0.000000,0.209358,0.790642,0.276661,0.000000,0.459750,0.250270,127,0,16,2.655126,111,195.866300,0.000000,181.118748,177.044840,0.100762,0.000000,0.051000,0.103647,0,0.00,NaN,4,0,125,24,False,e,15FM24,1.000,2,1,0.250,1,3,1,1,19,11,9,4,0.250,0.250,0.250,3,126,24,False,e,15FM24,1699,1,0,1,0,49597,171,48924,502,2239.671620,0,125,24,False,e,15FM24
2,5,120.252,91.824,1.908,14.700,75.216,0.020779,0.160089,0.819132,0.216056,0.190800,0.294000,0.206071,425,10,50,3.534245,365,203.608382,43.792914,266.760397,178.983519,0.067047,0.030000,0.070560,0.064357,8,2.00,627.670007,4,0,123,24,False,c,15FM24,0.750,4,1,0.500,4,4,1,1,14,12,9,3,0.500,0.125,0.125,4,124,24,False,c,15FM24,2021,1,0,1,0,125074,0,27119,97955,5165.991806,0,123,24,False,c,15FM24
3,13,286.536,220.632,9.408,22.944,188.280,0.042641,0.103992,0.853367,0.199667,0.218791,0.222757,0.196330,1105,43,103,3.856409,959,164.081059,95.722974,113.842406,152.276423,0.059696,0.045913,0.052200,0.057393,22,1.10,575.071324,9,4,124,24,False,c,15FM24,0.500,1,1,0.500,1,4,1,1,14,12,9,3,NaN,0.125,NaN,0,123,24,False,c,15FM24,10035,3,0,1,2,289551,50,16641,272860,8162.465534,4,124,24,False,c,15FM24
4,7,23.652,18.996,6.660,7.968,4.368,0.350600,0.419457,0.229943,0.237450,0.256154,0.215351,0.256941,80,26,37,3.382378,17,215.282084,110.765805,132.050341,240.901809,0.058937,0.051692,0.051273,0.056400,13,3.25,523.865325,4,0,129,24,False,s,15FM24,0.875,6,1,0.500,2,3,3,1,4,16,19,1,0.500,0.375,0.375,1,130,24,False,s,15FM24,236674,0,0,0,0,-1474040880184,-1474041169363,212948,76231,11077.388050,1,130,24,False,s,15FM24
5,19,327.636,245.184,19.116,39.060,187.008,0.077966,0.159309,0.762725,0.211366,0.265500,0.215801,0.206183,1160,72,181,3.540514,907,166.017369,116.502241,138.238493,154.943233,0.071141,0.066375,0.107374,0.060365,44,2.20,367.932505,12,1,130,24,False,s,15FM24,0.500,1,1,0.500,1,1,1,1,4,16,19,1,NaN,0.375,NaN,0,129,24,False,s,15FM24,3914,1,0,0,1,-1474041058073,136,-1474041064902,6693,5225.228176,0,129,24,False,s,15FM24
6,0,263.472,205.752,0.000,13.236,192.516,0.000000,0.064330,0.935670,0.208462,0.000000,0.210095,0.208351,987,0,63,3.746129,924,154.459433,0.000000,102.157191,153.228545,0.058540,0.000000,0.041806,0.058532,0,0.00,NaN,10,2,128,24,False,n,15FM24,0.875,5,1,0.750,1,2,2,1,10,5,9,2,0.750,0.500,0.500,2,128,24,False,n,15FM24,10721,2,0,0,2,-1474040351958,0,-1474040576914,224956,7792.194569,2,128,24,False,n,15FM24
7,0,17.436,14.208,0.000,1.452,12.756,0.000000,0.102196,0.897804,0.225524,0.000000,0.207429,0.227786,63,0,7,3.613214,56,173.781138,0.000000,75.088177,179.535866,0.052065,0.000000,0.042000,0.051840,0,0.00,NaN,4,0,127,24,False,n,15FM24,0.500,1,1,0.750,1,1,1,1,10,5,9,2,NaN,0.500,NaN,0,127,24,False,n,15FM24,2093,0,0,0,0,18039,0,700,17339,1955.446455,0,127,24,False,n,15FM24
8,2,14.364,11.496,0.804,1.584,9.108,0.069937,0.137787,0.792276,0.273714,0.268000,0.316800,0.267882,42,3,5,2.923977,34,251.643426,0.000000,42.603710,174.257332,0.069951,0.000000,0.036000,0.066667,3,0.75,516.187591,4,0,161,20,False,e,21MDC20,0.500,1,1,0.500,1,1,1,1,13,1,7,4,NaN,0.500,NaN,0,161,20,False,e,21MDC20,1118,1,0,0,1,12395,1675,3446,7274,4454.201258,0,161,20,False,e,21MDC20
9,15,500.352,40

In [12]:
frameQuestionnaire[:5]

,M1,M18,M19,M2,M20,M21,M22,M23,M24,M25,M26,M29,M2country,M30,M30country,country,experimentNumber,orderNumber,training,type,userId
0,0.500,1,1,0.25,1,1,1,1,19,11,9,4,NaN,0.250,NaN,0,125,24,False,e,15FM24
1,1.000,2,1,0.25,1,3,1,1,19,11,9,4,0.25,0.250,0.250,3,126,24,False,e,15FM24
2,0.750,4,1,0.50,4,4,1,1,14,12,9,3,0.50,0.125,0.125,4,124,24,False,c,15FM24
3,0.500,1,1,0.50,1,4,1,1,14,12,9,3,NaN,0.125,NaN,0,123,24,False,c,15FM24
4,0.875,6,1,0.50,2,3,3,1,4,16,19,1,0.50,0.375,0.375,1,130,24,False,s,15FM24


In [13]:
frameInteraction[:5]

,M14,M15,M15m,M15o,M15t,M16,M16m,M16o,M16t,M17,country,experimentNumber,orderNumber,training,type,userId
0,16073,1,0,1,0,282589,0,53137,229452,5999.696358,3,126,24,False,e,15FM24
1,1699,1,0,1,0,49597,171,48924,502,2239.671620,0,125,24,False,e,15FM24
2,2021,1,0,1,0,125074,0,27119,97955,5165.991806,0,123,24,False,c,15FM24
3,10035,3,0,1,2,289551,50,16641,272860,8162.465534,4,124,24,False,c,15FM24
4,236674,0,0,0,0,-1474040880184,-1474041169363,212948,76231,11077.388050,1,130,24,False,s,15FM24
